# Predicting the Survival of New Businesses in Vancouver

In [ ]:
# preprocessing
from DataPreprocess import *
from DataFetch import fetch_business_license, fetch_econ_indicators
from EDA import *
import pandas as pd
import numpy as np

# viz
import matplotlib.pyplot as plt
import altair as alt
alt.data_transformers.enable("vegafusion")

# reduce file size
alt.renderers.enable('svg')

#sklearn tool
from sklearn.model_selection import train_test_split
from sklearn.compose import make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import cross_validate

# models
from Modeling import transform, split_x_y
from sklearn.svm import SVC
from sklearn.naive_bayes import GaussianNB, BernoulliNB
from sklearn.linear_model import LogisticRegression

## Summary
This analysis aims to predict the survival of new businesses in Vancouver by examining various economic and demographic factors. Using datasets from the City business license registry (City of Vancouver 2023) and other external sources (Statistics Canada 2023), we explore the influence of location, industry, and economic indicators on business survival.

## Introduction
Vancouver's dynamic business landscape is influenced by various factors, including economic cycles, demographic shifts, and urban planning. Predicting the survival of new businesses in this environment is crucial for policymakers and entrepreneurs alike. This project seeks to answer: "Can the survival of a new business in Vancouver be predicted?" We utilize datasets from Vancouver's open data portal and integrate external data such as economic indicators and census data.
We are using the following Python packages to perform the analysis: Pandas (McKinney 2010), altair (VanderPlas, 2018), scikit-learn (Pedregosa et al. 2011).

### Dataset Description
The primary dataset comes from the City of Vancouver's business license registry, updated as businesses are licensed, renew, or terminate. This dataset is enriched with external data, including economic indicators.




## Loading dataset & Preprocessing

In [ ]:
# Fetch data by urls --> already modulized in DataFetch.py
business = fetch_business_license()
raw_econ_index_data_dict = fetch_econ_indicators()

### Business Lisence data
#### Clean-up
- Drop rows where `ExpiredDate` and `IssuedDate` are NA.
- Transform `ExpiredDate` and `IssuedDate` to date.
- Calculate the survival interval of each company, which is the difference between the maximum of ExpiredDate and the minimum of IssuedDate.
- Keep only the newest issued record of each company.
- Filter to keep those records where the latest `ExpiredDate` is before or equal to year 2022 because for those licenses issued in year 2023, the dafault `ExpiredDate` are `2023-12-31` and we cannot know whether it would survive until then.

#### Response Variable for Classification: survival_status
- Since about half of the companies in our dataset can survive over 2 years, to balance the amount of True & False, we set the threshold to 2 years 

In [ ]:
# Drop rows where ExpiredDate and IssuedDate are NA
business = business_datacleaning(business = business, survival_threshold = 365 * 2)

### Macroeconomics Data
- Create a column `REF_YEAR` representing the year of `REF_DATE`
- Keep rows where `North American Industry Classification System (NAICS) == 'All industries'`, since it is time-consuming to manually map the `BusinessType` in business license dataset to the related industries, we will merely consider the overall GDP performance in this project.
- Keep rows where `REF_YEAR >= 2012`
- Keep columns `REF_YEAR` and `VALUE`

In [ ]:
econ = econ_datacleaning(raw_econ_index_data_dict)

### Combine business lisence and macroeconomics data
- Map the yearly GDP value to the first lisence issued year of each company (the year when a company starts it business).

In [ ]:
business_econ = merge_business_econ_by_year(business, econ)
business_econ.head()

## EDA & Visualization

Overall Target: To look at which of the features might be useful to predict the survival status, we plotted the distributions of each predictor from the dataset and coloured the distribution by class (failed to survive more than 2 yrs: green, and survived for more than 2 yrs: orange). In doing this, what we aim at is to omit features of which both the binary classes have similar patterns. In that way, it means that these features do have the power to tell the two classes apart and fit their values into each of them.

In [ ]:
business_econ.info()

In [ ]:
business.describe(include='all')

### Numeric Features

We start with numeric features. By deleting features that make no sense or have limited value to dig into, we picked 6 numeric features to implement EDA on. 

With the generated figures, we found out that patterns of all the numeric features of the two classes look very similar in the pattern. Thus, we choose to omit all of these from our model.


In [ ]:
numeric_features = ['GDPValue', 'ConsumerPriceValue', 'EmploymentValue', 'InvestmentConstructionValue'] 
# Save Numberofemployees and FeePaid for later due to their large variance

numeric_feature_visualization(business_econ, numeric_features)

In [ ]:
#Plot number of employees of each company

large_variance_numeric_feature_visualization(business_econ, 'NumberofEmployees')

In [ ]:
#Plot fee paid of each company

large_variance_numeric_feature_visualization(business_econ, 'FeePaid')

### Categorical Features

For categorical features we generated histograms to see frequency of observations of both classes. 

The two histograms indicate an underlying pattern where the two features could have an influence on the target, with the similar spread of frequencies.

In [ ]:
business_econ['City'].value_counts() # With significantly large proportion of data in Vancouver, we would focus our research only on Vancouver
business_econ['Province'].value_counts() # Since most of the data are in BC Province, we would look into records in BC.

categorical_features = ['LocalArea', 'BusinessType'] 

In [ ]:
#Plot frequency of company locations

categorical_feature_visualization(business_econ, 'LocalArea')

In [ ]:
business_econ['BusinessType'].value_counts()

In [ ]:
#Plot 20 most frequent types of business

varianced_categorical_feature_visualization(business_econ, 'BusinessType')

### Analysis

We have used Logistic Regression and BernoulliNB for predicting business survival in Vancouver due to the nature of the data. 

##### Why Logistic Regression and BernoulliNB?
Logistic Regression is effective when the outcome is binary, making it appropriate for predicting whether a business survives or not. Easier interpretability of the model results is another reason why we chose Logistic Regression. It's a linear model that provides coefficients for each predictor variable, making it easy to interpret the impact of each variable on the predicted outcome. This can be crucial for understanding the economic and demographic factors influencing business survival. 

BernoulliNB, a variant of Naive Bayes, accommodates binary outcomes, aligning with the nature of the task where businesses either survive or fail. It excels in handling categorical features and is effective in scenarios with sparse data, making it well-suited for the diverse and potentially sparse economic and demographic factors influencing business longevity in the city.

##### Train Test Split
We are using 70% of our data as training data and the remaining 30% is used as test data.


##### Results
Logistic Regression is performing better and we are getting a cross-validation accuracy of ~80% (79.2%) on whether a business will survive or not. BernoulliNB has slightly lower cross-validation accuracy of 74.6%.

In [ ]:
# As the EDA part shows, with significantly large proportion of data in Vancouver, we would focus our research only on Vancouver
business = business[business['City'] == 'Vancouver']
business_econ = merge_business_econ_by_year(business, econ)
business_econ.head()

In [ ]:
train_df, test_df = train_test_split(business_econ, test_size=0.3, random_state=123)

features = {
    'word_features': ['BusinessType'],
    'categorical_features': ['City', 'LocalArea'],
    'numeric_features': ['NumberofEmployees', 'FeePaid', 
                        'GDPValue', 'ConsumerPriceValue', 'EmploymentValue', 'InvestmentConstructionValue']
}

X_train, y_train = split_x_y(train_df, **features)
X_test, y_test = split_x_y(test_df, **features)

X_train_transformed = transform(X_train, **features)
X_test_transformed = transform(X_test, **features)

In [ ]:
bnb = BernoulliNB()
pd.DataFrame(cross_validate(bnb, X_train_transformed, y_train, cv=10, return_train_score=True))

In [ ]:
logreg = LogisticRegression(random_state=123, max_iter=1000)
pd.DataFrame(cross_validate(logreg, X_train_transformed, y_train, cv=10, return_train_score=True))

In [ ]:

logreg.fit(X_test_transformed, y_test)
logreg.score(X_test_transformed, y_test)

#### Conclusion and Improvements

The Logistic Regression model gives a decent accuracy of ~80% here and can be used as an assistive model for making decisions on whether business licence will be renewed or not. 

We can further improve the model results:

- Trying out more complex models like Random Forest, Neural Networks etc. (which are currently out of our MDS syllabus scope as of now).
- By combining other economic and socio-economic factors in our dataset

## References

City of Vancouver. 2023. 'Business Licences Dataset.' Vancouver Open Data. https://opendata.vancouver.ca/explore/dataset/business-licences/information/?disjunctive.status&disjunctive.businesssubtype&refine.folderyear=23

Statistics Canada. 2023. https://www150.statcan.gc.ca/n1/en/type/data?MM=1

McKinney, Wes. 2010. “Data Structures for Statistical Computing in Python.” In Proceedings of the 9th Python in Science Conference, edited by Stéfan van der Walt and Jarrod Millman, 51–56.

VanderPlas, J. et al., 2018. Altair: Interactive statistical visualizations for python. Journal of open source software, 3(32), p.1057.

Pedregosa, F. et al., 2011. Scikit-learn: Machine learning in Python. Journal of machine learning research, 12(Oct), pp.2825–2830.